This notebook shows how a sound model works.

In [ ]:
import datetime
import numpy as np
from matplotlib import pyplot as plt

from utils.physics.sound_model.spherical_sound_model import HomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel
from utils.physics.geodesic.distance import distance_point_point

Propagation time computation

In [ ]:
# propagation time

pos_1 = [-25, 10]  # lat, lon format (in degrees)
pos_2 = [-30, 15]
# note: to get the position of a station s, just do s.get_pos()

# uncomment the wanted sound model
sound_model = GridSoundModel([f"../../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)])
#sound_model = HomogeneousSoundModel(sound_speed=1480)


propagation_time = sound_model.get_sound_travel_time(pos_1, pos_2, datetime.datetime.now())
print(f"Propagation time between {pos_1} and {pos_2}: {propagation_time:.2f}s")
print(f"(Distance is {distance_point_point(pos_1, pos_2)/1_000:.2f} km)")

Localization (given 4 detection time and positions, find the source)

In [ ]:
# location
detection_pos_1 = [5, 0]
detection_pos_2 = [-5, 0]
detection_pos_3 = [0, 5]
detection_pos_4 = [0, -5]
detection_positions = [detection_pos_1, detection_pos_2, detection_pos_3, detection_pos_4]

detection_time_1 = datetime.datetime(2020, 1, 1, 0, 0, 0)
detection_time_2 = detection_time_1
detection_time_3 = detection_time_1
detection_time_4 = detection_time_1
detection_times = [detection_time_1, detection_time_2, detection_time_3, detection_time_4]

inversion = sound_model.localize_common_source(detection_positions, detection_times)
print(f"Found source position {inversion.x[1:]} with an emission {inversion.x[0]:.2f} s before the first detection. Cost was {inversion.cost:.2f}.")

Now do this same process but adding random noise on both detection times and sound velocities, to obtain a Monte-Carlo sampling of the position.

In [ ]:
# MC uncertainty estimate
pos = sound_model.MC_error_estimation(1000, pick_sigma=1.5, velocity_sigma=0.5, sensors_positions=detection_positions, detection_times=detection_times)
pos = np.array(pos)
plt.scatter(pos[:,2], pos[:,1])